In [1]:
from src.algo.boost.ada import load_xingye_data , AdaBoost
import pandas as pd

factor_data = load_xingye_data()
MDTs = factor_data['date'].unique()   
windows_len = 24 

ic_dfs = []
start_idx = len(MDTs) - 2


In [2]:
import cProfile
idx = len(MDTs) - 2

with cProfile.Profile() as t:
    input_df = AdaBoost.df_input(factor_data , idx , windows_len)

df = pd.DataFrame(getattr(t , 'getstats')(), 
             columns=['full_name', 'ncalls', 'ccalls', 'tottime', 'cumtime' , 'caller'])

Train: 20210930 - 20230831 , Test : 20230928


In [4]:
start_idx = len(MDTs) - 2
for idx in range(max(windows_len , start_idx) , len(MDTs)):
    input_df = AdaBoost.df_input(factor_data , idx , windows_len)
    ada = AdaBoost(**input_df)
    ic_dfs.append(ada.fit(silence = True).calc_ic())
df = pd.concat(ic_dfs)
print(df)

Train: 20210930 - 20230831 , Test : 20230928
Train: 20211029 - 20230928 , Test : 20231031
                ic   rankic
20230928  0.019810  0.07644
20231031  0.061668  0.13238


In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os , torch
from src.algo.boost.ada import AdaBoost

factor_data = pd.read_feather('./data/TreeData/CombStdByZXMkt_All_TrainLabel.feather') # 训练集，带Label
factor_data['date'] = factor_data['date'].astype(str).str.replace('-','').astype(int)
v = factor_data['StockID'].astype(str).str.slice(0, 6).replace({'T00018' : '600018'})
v = v.where(v.str.isdigit() , '-1').astype(int)
factor_data['secid'] = v
factor_data = factor_data.drop(columns=['ZX','mktVal','mktValRank','StockID']).sort_values(['secid','date'])

MDTs = factor_data['date'].unique()   # 测试集日期-完整
windowsLen = 24 # 回望过去的月数


In [2]:
def parse_input_df(df : pd.DataFrame , training : bool , n_bins : int , label_column = 'y_label') -> pd.DataFrame:
    x = df.drop(columns=[label_column])
    y = df[label_column]

    if y.isnull().sum() / len(y) > 0.2: print('Error: label missing ratio > 0.2')
    
    df = (x.groupby('date').rank(pct = True) * n_bins).clip(upper=n_bins - 1).join(y)
    if training: 
        #missing = df.isnull().sum() / len(df)
        #df = df.loc[:, missing[missing < 0.2].index.values] # .fillna(0.0)
        df = df.loc[:, df.isnull().sum() / len(df) < 0.2] # .fillna(0.0)
        df = df[df[label_column] != 0].dropna(subset=[label_column])
    return df

In [4]:
MDTs

array([20050131, 20050228, 20050331, 20050429, 20050531, 20050630,
       20050729, 20050831, 20050930, 20051031, 20051130, 20051230,
       20060125, 20060228, 20060331, 20060428, 20060531, 20060630,
       20060731, 20060831, 20060929, 20061031, 20061130, 20061229,
       20070131, 20070228, 20070330, 20070430, 20070531, 20070629,
       20070731, 20070831, 20070928, 20071031, 20071130, 20071228,
       20080131, 20080229, 20080331, 20080430, 20080530, 20080630,
       20080731, 20080829, 20080926, 20081031, 20081128, 20081231,
       20090123, 20090227, 20090331, 20090430, 20090527, 20090630,
       20090731, 20090831, 20090930, 20091030, 20091130, 20091231,
       20100129, 20100226, 20100331, 20100430, 20100531, 20100630,
       20100730, 20100831, 20100930, 20101029, 20101130, 20101231,
       20110131, 20110228, 20110331, 20110429, 20110531, 20110630,
       20110729, 20110831, 20110930, 20111031, 20111130, 20111230,
       20120131, 20120229, 20120330, 20120427, 20120531, 20120

In [3]:
ic_dfs = []
start_idx = 179
for idx in range(max(windowsLen - 1 , start_idx) , len(MDTs)-1):
    idtEnd = MDTs[idx]
    idtStart = MDTs[idx - windowsLen + 1]
    idtTest = MDTs[idx + 1]

    print(f'Train: {idtStart} - {idtEnd} , Test : {idtTest}')
    train_data = factor_data.loc[(factor_data['date'] >= idtStart) & (factor_data['date'] <= idtEnd),:].\
        drop(columns=['nextRtnM']).rename(columns={'nextRtnM_Label':'y_label'}).set_index(['date', 'secid'])
    test_data = factor_data.loc[factor_data['date'] == idtTest,:].\
        drop(columns=['nextRtnM_Label']).rename(columns={'nextRtnM':'y_label'}).set_index(['date', 'secid'])
    train = parse_input_df(train_data , True , 20)
    test = parse_input_df(test_data , False , 20).loc[:,train.columns.values]
    ada = AdaBoost(train = train , test = test)
    ic_dfs.append(ada.fit(silence = True).calc_ic())
df = pd.concat(ic_dfs)
df


Train: 20180131 - 20191231 , Test : 20200123


KeyboardInterrupt: 

In [2]:
train_data = factor_data.loc[(factor_data['date'] >= idtStart) & (factor_data['date'] <= idtEnd),:].\
    drop(columns=['nextRtnM']).rename(columns={'nextRtnM_Label':'y_label'}).set_index(['date', 'secid'])
test_data = factor_data.loc[factor_data['date'] == idtTest,:].\
    drop(columns=['nextRtnM_Label']).rename(columns={'nextRtnM':'y_label'}).set_index(['date', 'secid'])

def parse_input_df(df : pd.DataFrame , training : bool , n_bins : int , label_column = 'y_label') -> pd.DataFrame:
    x = df.drop(columns=[label_column])
    y = df[label_column]

    if y.isnull().sum() / len(y) > 0.2: print('Error: label missing ratio > 0.2')
    
    df = (x.groupby('date').rank(pct = True) * n_bins).clip(upper=n_bins - 1).join(y)
    if training: 
        #missing = df.isnull().sum() / len(df)
        #df = df.loc[:, missing[missing < 0.2].index.values] # .fillna(0.0)
        df = df.loc[:, df.isnull().sum() / len(df) < 0.2] # .fillna(0.0)
        df = df[df[label_column] != 0].dropna(subset=[label_column])
    return df

train = parse_input_df(train_data , True , 20)
test = parse_input_df(test_data , False , 20).loc[:,train.columns.values]

In [3]:
from src.algo.boost.ada import AdaBoost

ada = AdaBoost(train = train , test = test)

In [4]:
df = ada.fit().calc_ic()
df

Round: 1, Estimator Error: 0.4945843848750985, feature_idx: 129
Round: 6, Estimator Error: 0.49902970673461733, feature_idx: 42
Round: 11, Estimator Error: 0.49936663092986505, feature_idx: 135
Round: 16, Estimator Error: 0.49948198587216597, feature_idx: 50
Round: 21, Estimator Error: 0.49953335247401615, feature_idx: 87
Round: 26, Estimator Error: 0.4995382518292149, feature_idx: 90


,ic,rankic
20070131,-0.099588,-0.1118
